In [105]:
from datetime import timedelta
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from geopy.distance import vincenty

In [177]:
def distance_delta(row):
    lat = row.latitude
    long = row.longitude
    lat_1 = row.latitude_shift_1
    long_1 = row.longitude_shift_1
    
    if any(map(np.isnan, [lat, long, lat_1, long_1])):
        return np.nan
    return vincenty((lat, long), (lat_1, long_1)).miles

In [190]:
date_columns = ['timestamp']
df = pd.read_csv("./gps_dataset.csv", sep=";", parse_dates=date_columns)
df.sort_values('timestamp')
df['week_day'] = df.timestamp.dt.weekday_name
df['timestamp_delta'] = df.timestamp - df.timestamp.shift(1)
df['latitude_shift_1'] = df.latitude.shift(1)
df['longitude_shift_1'] = df.longitude.shift(1)
df['distance_delta'] = df.apply(distance_delta, axis=1)
df['is_new_walk_by_time'] = df.timestamp_delta > timedelta(minutes=5)
df['is_new_walk_by_distance'] = df.distance_delta > 0.1

In [191]:
df[df.is_new_walk_by_distance]

,timestamp,latitude,longitude,altitude,horizontalAccuracy,verticalAccuracy,speed,course,week_day,timestamp_delta,latitude_shift_1,longitude_shift_1,distance_delta,is_new_walk_by_time,is_new_walk_by_distance
61,2018-03-13 23:50:12,40.736822,-73.986770,28.897570,65.0,65.0,-3.6,-1.0,Tuesday,02:29:28,40.702374,-73.984167,2.380918,True,True
367,2018-03-14 21:59:52,40.737699,-73.989982,29.768972,65.0,65.0,-3.6,-1.0,Wednesday,08:38:07,40.735126,-73.990477,0.179434,True,True


In [192]:
df[df.is_new_walk_by_time]

,timestamp,latitude,longitude,altitude,horizontalAccuracy,verticalAccuracy,speed,course,week_day,timestamp_delta,latitude_shift_1,longitude_shift_1,distance_delta,is_new_walk_by_time,is_new_walk_by_distance
61,2018-03-13 23:50:12,40.736822,-73.986770,28.897570,65.0,65.0,-3.6,-1.0,Tuesday,02:29:28,40.702374,-73.984167,2.380918,True,True
158,2018-03-14 11:39:44,40.736866,-73.986781,29.113699,65.0,65.0,-3.6,-1.0,Wednesday,11:33:30,40.736848,-73.986697,0.004580,True,False
219,2018-03-14 12:59:28,40.737035,-73.987102,32.516083,30.0,30.0,0.0,-1.0,Wednesday,01:07:44,40.736935,-73.986922,0.011699,True,False
367,2018-03-14 21:59:52,40.737699,-73.989982,29.768972,65.0,65.0,-3.6,-1.0,Wednesday,08:38:07,40.735126,-73.990477,0.179434,True,True
459,2018-03-14 22:31:55,40.738591,-73.993763,28.245260,65.0,65.0,-3.6,-1.0,Wednesday,00:05:52,40.738621,-73.993879,0.006431,True,False
463,2018-03-14 23:20:57,40.738574,-73.993766,28.024649,65.0,65.0,-3.6,-1.0,Wednesday,00:48:34,40.738623,-73.993868,0.006332,True,False
465,2018-03-14 23:27:36,40.738694,-73.994015,27.731878,65.0,65.0,-3.6,-1.0,Wednesday,00:06:30,40.738621,-73.993869,0.009170,True,False
508,2018-03-15 11:37:20,40.736814,-73.986764,28.921131,65.0,65.0,-3.6,-1.0,Thursday,12:01:16,40.736845,-73.986755,0.002191,True,False


In [198]:
df[df.is_new_walk_by_distance == True and df.is_new_walk_by_time.bool() == True]

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().